In [ ]:
from tutti_etl.databases import redshift_db
from datetime import date
from typing import List

#So we limit the data to all the public ads from tutti.ch, which have 3000 or more pageviews and were created in the german language

sql = """
    select
        region,
        ad_top_category,
        subject as ad_title
    from
        public.ads
    where
        pageviews >= 3000
        and lang = 'de'
"""

#FYI:the password for the database was changed for security reason on the final file

db = redshift_db(host='dwh.tuttidataformichiel.ch', port=5439,
                 user='tutti_read_only', password='can not be public on GitHub', db_name='dwh_scm_ch')

data = db.execute(sql=sql)


In [239]:
#step 1: opening a file 
file=open('most_common_ad_regions.txt', 'w')

#step 2: determine the first element (0) of the tuple -> region 
region=[regions[0] for regions in data]

#step 3: Change all the first letters of the region to capital letter (visual purpose)
region=[region_capital.title() for region_capital in region]

#step 4: create a counter
import collections
counter=collections.Counter(region)

#step 5: print all the 24 regions in the order of their counts
#print every element per line 

most_common = counter.most_common(24)

for common in most_common:
    file.write('Canton:'+ '\033[1m'+common[0]+'\033[0m'+';'+'  Count:'+ str(common[1])+'\n')

file.close()


In [240]:
#step 1: opening a file 
file=open('most_common_ad_categories.txt', 'w')

#step 2: determine the second element (1) of the tuple -> ad_top_category
ad_top_category=[categories[1] for categories in data]

#step 3: Change all the first letters of the ad_top_category to capital letter (visual purpose)
ad_top_category=[category_capital.title() for category_capital in ad_top_category]

#step 4: create a counter
import collections
counter=collections.Counter(ad_top_category)

#step 5: print all the 23 top categories (in the german language) 
#print every element on a new line 

most_common = counter.most_common(30)
for common in most_common:
    file.write('Category:'+ '\033[1m'+common[0]+'\033[0m'+';'+' Count:'+ str(common[1])+'\n')

file.close()


In [241]:
#step 1: opening a file 
file=open('most_common_ad_titles.txt', 'w')

#step 1: determine the third element (2) of the tuple -> ad title 
ad_title=[title[2] for title in data]

#step 2: Change all the first letters of the ad_title words to capital letter
#We do this here becasue otherwise we would have some duplicates (e.g Appartamento and appartamento)

ad_title=[ad_title_capital.title() for ad_title_capital in ad_title]

#step 3: define 'stopwords' -> in this context: strings, which are only one character or do not deliver any valid content
#We do this because we want to get relevant results 

stopwords=['-', '/', '', ' ', 'A', 'E', 'Zu', 'In', 'Di', 'Per', 'Con', 'Mit', 'Mfk', '3.5', 'Tdi', 'Und', 'Ab', '2', 'Für', '4', '2.5', 'Da', '4.0', '3.5', '2.0', '3', '4.5', 'Auf', 'Im', '5', '1.9']

#step 4: create a counter
import collections
counter=collections.Counter()

#step 5: iterate over each ad title with a for loop -> split it by white space -> feed the resulting list into the counter 

for title in ad_title:
    itemWords=title.split(' ')
    for w in itemWords:
        if w not in stopwords:
            counter[w] +=1
    
#step 6: print the the 25 most common ad_titles
most_common = counter.most_common(25)

for common in most_common:
    file.write('Title word:'+ '\033[1m'+common[0]+'\033[0m'+';'+' Count:'+ str(common[1])+'\n')

file.close()